I'm  using this Jupyter notebook to learn how to use the [Open plan of the WattTime API](https://api.watttime.org/plans/).  At first, I thought that I could make use of [WattTime/watttime-python-client: Python client library for the WattTime API](https://github.com/WattTime/watttime-python-client) for this purpose but then learned that this Python library is designed for users of the Pro plan.  Some of the code here could possibly be integrated into the `watttime-python-client`.

**Note: some cell outputs with data coming from the API have been cleared**

In [1]:
%matplotlib inline

/Users/raymondyee/anaconda/envs/myenv/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# storing credentials in settings.py

from settings import (WATTTIME_API_TOKEN, WATTTIME_USER, WATTTIME_PASSWORD)

import os
import requests
import arrow, datetime

import numpy as np
import datetime as dt
import time

import matplotlib.pyplot as plt
import matplotlib.dates as md

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

# starter code for a database to cache Watttime data
# I'm in the process of figuring out the relevant term of services for caching such data

import watttime_db

# obtaining token

To use the Watttime API, one needs to obtain a token

In [3]:
# must make a call to https://api.watttime.org/api/v1/obtain-token-auth/

import urlparse

def obtain_token(username, password):
    
    base_url = 'https://api.watttime.org'
    endpoint = urlparse.urljoin(base_url, '/api/v1/obtain-token-auth/')
    
    payload = {'username': username, 
               'password': password
             }

    result = requests.post(endpoint, data=payload)
    return result.json()

# confirm that the procedure used to convert username/password to token is the same as WATTTIME_API_TOKEN

( obtain_token(WATTTIME_USER, WATTTIME_PASSWORD).get('token')
  ==
  WATTTIME_API_TOKEN
)

True

# validating token

In [4]:
# one way of testing whether a given token is ok

def validate_token (token):
    
    base_url = 'https://api.watttime.org'
    endpoint = urlparse.urljoin(base_url, '/api/v1/datapoints/')
    
    headers = {
        'Authorization': 'Token {}'.format(token)
    }
    
    result = requests.head(endpoint, headers=headers)
    return result.status_code == 200


In [5]:
validate_token(WATTTIME_API_TOKEN)

True

# balance authority

e.g., https://api.watttime.org/api/v1/balancing_authorities/?loc={"type":"Point","coordinates":[-122.272778,37.871667]} 


In [6]:
import json

def balance_authority(lat, lon):
    
    base_url = 'https://api.watttime.org'
    endpoint = urlparse.urljoin(base_url, '/api/v1/balancing_authorities/')
    
    loc_info = json.dumps({
            'type': 'Point',
            'coordinates': [lon, lat]
        })
    
    
    params = {
        'loc': loc_info
    }
    
    result = requests.get(endpoint, params=params)
    return result.json()


In [7]:
# look up the balance authority for Berkeley, CA

balance_authority(lat=37.871667, lon=-122.272778)

[{u'abbrev': u'CAISO',
  u'ba_type': u'ISO',
  u'link': u'http://oasis.caiso.com/',
  u'name': u'California Independent System Operator',
  u'notes': u' ',
  u'states': [u'CA'],
  u'url': u'https://api.watttime.org/api/v1/balancing_authorities/CAISO/'}]

# generation mix

Some documentation:

```
ba	An abbreviation for a balancing authority. Options can be found at the 'balancing_authorities' endpoint. e.g., ba=ISONE	query	string
start_at	
Minimum timestamp (inclusive). e.g., start_at=2014-02-20 or start_at=2014-02-20T16:45:30-0800 or start_at=2014-02-20T16:45:30-08:00	query	string
end_at	
Maximum timestamp (inclusive). e.g., end_at=2014-02-20 or end_at=2014-02-20T16:45:30-0800 or end_at=2014-02-20T16:45:30-08:00	query	string
page_size	
Number of data points to return on each page. default is page_size=100, max is page_size=1000.	query	integer
freq	
Time series frequency. Options are '5m', '10m', '1hr', 'n/a'. e.g., freq=1hr	query	string
market
```

```
ba filters by place (see tutorial)
start_at and end_at filter by time (formatted like 2015-10-20T16:45:30-08:00)
market filters by data type (use RT5M to get past data from the real-time 5 minute market, or DAHR to get forecasted data from the day-ahead hourly market)
page_size sets the number of data points that are returned in a single query

```

In [8]:
# display now
arrow.get(datetime.datetime.utcnow()).isoformat()

'2016-12-07T19:55:41.100643+00:00'

In [9]:
# example of a raw requestion

def filter_dict_None(d):
    return dict([(k,v) for (k,v) in d.items() if v is not None])

# a first pass at getting the generation mix

def generation_mix_0(token, ba, start_at, end_at, page_size=None, market='RT5M', **kwargs):
    """
    * ba filters by place (see tutorial)
    * start_at and end_at filter by time (formatted like 2015-10-20T16:45:30-08:00)
    * market filters by data type (use RT5M to get past data from the real-time 5 minute market, 
        or DAHR to get forecasted data from the day-ahead hourly market)
    * page_size sets the number of data points that are returned in a single query
    """
    base_url = 'https://api.watttime.org'
    url = urlparse.urljoin(base_url, '/api/v1/datapoints/')
    
    headers = {
        'Authorization': 'Token {}'.format(token)
    }
    
    params = filter_dict_None({
        'ba': ba,
        'start_at': arrow.get(start_at).isoformat(),
        'end_at': arrow.get(end_at).isoformat(),
        'page_size': page_size,
        'market': market
    })
    

    r = requests.get(url, params=params)
    return r.json()


In [10]:
import datetime, pytz, arrow

now = arrow.utcnow()
end_at = now

# pick a start time a given time in the past
start_at = now.replace(years=-10)

r = generation_mix_0(token = WATTTIME_API_TOKEN, ba='CAISO', start_at=start_at, end_at=end_at)


In [11]:
# how many records are there 

r['count'], r['next'], r['previous']

(127880,
 u'https://api.watttime.org/api/v1/datapoints/?ba=CAISO&end_at=2016-12-07T19%3A55%3A41.139084%2B00%3A00&market=RT5M&page=2&start_at=2006-12-07T19%3A55%3A41.139084%2B00%3A00',
 None)

Rewriting method to get generation mix as a generator. 

In [12]:
def generation_mix(token, ba, start_at, end_at, page_size=100, market='RT5M', **kwargs):
    """
    * ba filters by place (see tutorial)
    * start_at and end_at filter by time (formatted like 2015-10-20T16:45:30-08:00)
    * market filters by data type (use RT5M to get past data from the real-time 5 minute market, 
        or DAHR to get forecasted data from the day-ahead hourly market)
    * page_size sets the number of data points that are returned in a single query
    """
    base_url = 'https://api.watttime.org'
    url = urlparse.urljoin(base_url, '/api/v1/datapoints/')
    
    headers = {
        'Authorization': 'Token {}'.format(token)
    }
    
    params = {
        'ba': ba,
        'start_at': arrow.get(start_at).isoformat(),
        'end_at': arrow.get(end_at).isoformat(),
        'page_size': page_size,
        'market': market
    }
    
    more_pages = True
    
    while more_pages:
    
        r = requests.get(url, params=params)
        results = r.json()['results']
        
        for result in results:
            yield result
        
        url = r.json()['next']
        more_pages = url is not None


In [13]:
# some number of minutes before to now

import datetime, pytz, arrow

now = arrow.now()
end_at = now
start_at = now.replace(minutes=-6000)

results = list(generation_mix(token = WATTTIME_API_TOKEN, ba='CAISO', start_at=start_at, end_at=end_at))

# sidebar: how to use django cache

```Python
from django.core.cache.backends.locmem import LocMemCache
cache = LocMemCache('rycache', {})
```

# Display CAISO data for a recent range

In [14]:
# sort the results and display carbon emission rate

results = sorted(results, key = lambda s: s['timestamp'], reverse=True)
df = DataFrame(
    [(arrow.get(result['timestamp']).datetime, result['carbon'])  for result in results],
    columns = ['ts', 'carbon']
)
len(df)

599

In [ ]:
# http://stackoverflow.com/questions/4090383/plotting-unix-timestamps-in-matplotlib

plt.subplots_adjust(bottom=0.2)
plt.xticks( rotation=25 )
ax=plt.gca()
xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
ax.xaxis.set_major_formatter(xfmt)


plt.plot(df['ts'], df['carbon'])

# Database

I'm experimenting with caching data so that I can do more elaborate analysis

In [16]:
# initialize

db_fname = 'watttime.db'
if not os.path.exists(db_fname):
    engine = watttime_db.create_tables('watttime.db')

In [17]:
session = watttime_db.create_session(db_fname)

In [18]:
session.query(watttime_db.Datapoint).all()

[<Datapoint(ba='CAISO' timestamp='2016-11-01T16:20:00Z')>,
 <Datapoint(ba='CAISO' timestamp='2016-11-29T23:10:00Z')>,
 <Datapoint(ba='CAISO' timestamp='2016-12-07T16:50:00Z')>,
 <Datapoint(ba='CAISO' timestamp='2016-12-07T18:20:00Z')>,
 <Datapoint(ba='CAISO' timestamp='2016-12-07T18:40:00Z')>]

In [ ]:
# try to store a datapoint

result = results[0]
result

In [20]:
import re

def datapoint_id_from_url(url):
    g = re.search(r'([^\/]*)\/?$', url)
    if g is not None:
        return int(g.group(1))
    else:
        return None

In [21]:
dp = watttime_db.Datapoint(
    id = datapoint_id_from_url(result['url']),
    ba = result['ba'],
    carbon = result['carbon'],
    created_at = result['created_at'],
    freq = result['freq'],
    market = result['market'],
    timestamp = result['timestamp'],
    url = result['url'],
    updated=arrow.utcnow().isoformat()
)

dp.genmixes = [watttime_db.Genmix(
   fuel = g['fuel'],
   gen_mw = g['gen_MW']
) for g in result['genmix']]

In [22]:
session.add(dp)
session.commit()

# Watttime client

This code will not work for me because I'm on the Open plan not the Pro plan.

In [23]:
from watttime_client.client import WattTimeAPI
mytoken = WATTTIME_API_TOKEN
client = WattTimeAPI(token=mytoken)

No handlers could be found for logger "watttime_client.client"


In [24]:
from datetime import datetime
import pytz

timestamp = pytz.utc.localize(datetime(2016, 6, 1, 12, 30))
value = client.get_impact_at(timestamp, 'CAISO')

print(value)

KeyError: 'results'

# pyiso

I'm learning that a lot of the action in software development is actually around `pyiso`, a library for directly scraping power grid data sources, including my local ISO: [California Independent System Operator - Wikipedia](https://en.wikipedia.org/wiki/California_Independent_System_Operator).

* [Introduction — pyiso 0.3 documentation](https://pyiso.readthedocs.io/en/latest/intro.html)

* [WattTime/pyiso: Python client libraries for ISO and other power grid data sources.](https://github.com/WattTime/pyiso)

In [25]:
from pyiso import client_factory
caiso = client_factory('CAISO', timeout_seconds=60)

In [26]:
data = caiso.get_generation(latest=True)
df = DataFrame(data)
df.head()

,ba_name,freq,fuel_name,gen_MW,market,timestamp
0,CAISO,10m,renewable,1606.00,RT5M,2016-12-07 19:50:00+00:00
1,CAISO,10m,solar,5467.00,RT5M,2016-12-07 19:50:00+00:00
2,CAISO,10m,wind,131.00,RT5M,2016-12-07 19:50:00+00:00
3,CAISO,10m,other,30799.99,RT5M,2016-12-07 19:50:00+00:00


study the `get_generation` method for CAISO specifically. 

* [pyiso/caiso.py at 1a5334056d32ded43ce3797b5c5d7168c4d0c90f · WattTime/pyiso](https://github.com/WattTime/pyiso/blob/1a5334056d32ded43ce3797b5c5d7168c4d0c90f/pyiso/caiso.py#L89-L100)
* https://hyp.is/bMOFXrywEeaYU5vQBBnQ7Q/pyiso.readthedocs.io/en/latest/options.html:  yes to all of 
   * latest	
   * start_at / end_at
   * yesterday	
   * forecast ok

In [27]:
import datetime, pytz, arrow

now = arrow.now()
end_at = now
start_at = now.replace(minutes=-600)

data = caiso.get_generation(start_at= start_at, end_at=end_at)
df = DataFrame(data)
df.head()

,ba_name,freq,fuel_name,gen_MW,market,timestamp
0,CAISO,1hr,wind,352.05,DAHR,2016-12-07 15:00:00+00:00
1,CAISO,1hr,solar,1248.94,DAHR,2016-12-07 15:00:00+00:00
2,CAISO,1hr,wind,337.11,DAHR,2016-12-07 18:00:00+00:00
3,CAISO,1hr,solar,5557.67,DAHR,2016-12-07 18:00:00+00:00
4,CAISO,1hr,wind,357.46,DAHR,2016-12-07 17:00:00+00:00


Next steps:  figure out whether how to relate the generation mix data with the average carbon emissions available from the Watttime API. 